# User Interface

> Gradio interface for the chat application.

In [ ]:
#| default_exp ui

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
import gradio as gr
import tempfile
import datetime
import os
from typing import List, Tuple, Dict, Generator
from fastcore.basics import patch
from gradiochat.config import ChatAppConfig, ModelConfig
from gradiochat.app import BaseChatApp
from pathlib import Path

/home/jelle/code/gradiochat/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create the class for the User interface

A class that creates and manages a Gradio-based chat interface.

This class provides a web-based user interface for interacting with chat models.
It handles the display of messages, streaming of responses, and various UI elements
like buttons for sending messages, clearing chat history, and exporting conversations.

Attributes:

- app (BaseChatApp): The underlying chat application that handles message processing. It accepts an instance of the class `BaseChatApp` which is defined in the module `app.py`.
- interface (gr.Blocks, optional): The Gradio interface object once built.

The interface is built within this class with the `build_interface` method.

#### Import statement

```python
from gradiochat.ui import *
```

In [ ]:
#| export
class GradioChat:
    """Gradio interface for the chat application"""
    
    def __init__(self, app: BaseChatApp):
        """Initialize with a configured BaseChatApp"""
        self.app = app
        self.interface = None
    
    def respond(self, message: str, chat_history: List[Dict[str, str]]) -> Tuple[str, List[Tuple[str, str]]]:
        """Generate a response to the user message and update chat history"""
        # Store the current chat history in the app
        self.app.chat_history = chat_history
        
        # Generate response
        response = self.app.generate_response(message)
        
        # Update chat history
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": response})
        
        # Return empty message (to clear input) and updated history
        return "", chat_history
    
    def respond_stream(self, message: str, chat_history: List[Tuple[str, str]]) -> Generator[Tuple[str, List[Tuple[str, str]]], None, None]:
        """Generate a streaming response to the user message"""
        # Store the current chat history in the app
        self.app.chat_history = chat_history
        
        # Add user message to history with empty assistant response
        chat_history.append({"role": "user", "content": message})
        
        # Stream the response
        accumulated_text = ""
        for text_chunk in self.app.generate_stream(message):
            accumulated_text += text_chunk
            
            # Update the last assistant message
            updated_history = chat_history.copy()
            updated_history.append({"role": "assistant", "content": accumulated_text})
            
            # Yield empty message and updated history
            yield "", updated_history

#### Build the interface for the GradioChat class

Build and return the Gradio interface.

This method constructs the complete Gradio UI with all components including:
- App title and logo
- Chat display area
- Message input field
- Control buttons (Send, Clear)
- Export functionality
- System information display

The interface is configured according to the settings in the app's config.

Returns:
    gr.Blocks: The constructed Gradio interface object.

> **Learned: `@patch`**
> 
> This is a decorator used in `nbdev` to make it possible to spread the methods and properties of a class over multiple notebook cells. By using `@patch` and setting `self:<classname>` the `nbdev` style written code 'knows' to which class the method or property belongs.

In [ ]:
#| export
from datetime import datetime


@patch
def build_interface(self:GradioChat) -> gr.Blocks:
    """Build and return the Gradio interface"""
    with gr.Blocks(theme=self.app.config.theme, title=self.app.config.app_name) as interface:
        with gr.Row():
            # Left column for logo
            with gr.Column(scale=1):
                if self.app.config.logo_path:
                    gr.Image(value=self.app.config.logo_path,
                        show_label=False,
                        container=False,
                        show_download_button=False,
                        show_fullscreen_button=False,
                        height=80,
                        width=80)
                else:
                    gr.Image(value=None,
                        show_label=False,
                        container=False,
                        show_download_button=False,
                        show_fullscreen_button=False,
                        height=80,
                        width=80)
            with gr.Column(scale=4):
                # App title and description
                gr.Markdown(f"# {self.app.config.app_name}")
                if self.app.config.description:
                    gr.Markdown(self.app.config.description)
        
        # Chat interface
        chatbot = gr.Chatbot(
            height=500,
            label="Conversation",
            type="messages",
            editable=True,
            show_copy_button=True,
            show_copy_all_button=True)
        msg = gr.Textbox(
            placeholder="Type your message here...",
            label="Your message",
            lines=2
        )
        
        # Buttons
        with gr.Row():
            submit_btn = gr.Button("Send", variant="primary")
            clear_btn = gr.ClearButton([msg, chatbot], value="Clear chat")

        # Export functionality
        with gr.Accordion("Export Options", open=False):
            gr.Markdown("Select export options:")
            
            # Buttons for copying and downloading
            with gr.Row():
                download_btn = gr.DownloadButton(
                    label="Download as Markdown",
                    variant="secondary",
                    visible=True,
                    interactive=True
                )
        
        # System prompt and context viewer (collapsible)
        with gr.Accordion("View System Information", open=False):
            if self.app.config.show_system_prompt:
                gr.Markdown(f"### System Prompt\n{self.app.config.system_prompt}")
            
            if self.app.config.show_context and hasattr(self.app, 'context_text') and self.app.context_text:
                gr.Markdown(f"### Additional Context\n{self.app.context_text}")
        
        # Set up event handlers
        submit_btn.click(
            self.respond,
            inputs=[msg, chatbot],
            outputs=[msg, chatbot]
        )
        
        msg.submit(
            self.respond,
            inputs=[msg, chatbot],
            outputs=[msg, chatbot]
        )

            # Export event handlers
        def format_last_response(chat_history):
            if not chat_history:
                return "No conversation to export."
            msg = chat_history[-1]

            return f"# Response\n\n{msg['content']}"
        
        def format_full_conversation(chat_history):
            if not chat_history:
                return "No conversation to export."
            
            md_str = f"# {self.app.config.app_name} - Conversation\n\n"
            
            for msg in chat_history:
                role = msg["role"]
                content = msg["content"]
                if role == "user":
                    md_str += f"**👤 User:**\n{content}\n\n"
                elif role == "assistant":
                    md_str += f"**🤖 Assistant+**\n{content}\n\n"
                else:
                    md_str += f"**{role}:**\n{content}\n\n"
            return md_str
            
        # File download functionality
        def download_chat(chat_history):
            md_content = format_full_conversation(chat_history)
            temp_dir = tempfile.gettempdir()
            filename = f"conversation_{datetime.today().strftime('%Y-%m-%d')}.md"
            filepath = Path(temp_dir) / filename
            
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(md_content)
            
            os.chmod(filepath, 0o644)

            return filepath

        download_btn.click(
            fn=download_chat,
            inputs=[chatbot],
            outputs=[download_btn]
        )
            
        # Initialize with starter prompt if available
        if self.app.config.starter_prompt:
            chatbot.value = [{"role": "assistant", "content": self.app.config.starter_prompt}]
        
        self.interface = interface
        return interface

#### Method to launch the app that is instantiated from the class `GradioChat`

In [ ]:
#| export
@patch
def launch(self:GradioChat, **kwargs):
    """Launch the Gradio interface"""
    if self.interface is None:
        self.build_interface()
    
    return self.interface.launch(**kwargs)

## Create chat app from the class `GradioChat`

Launch the Gradio interface.

This method builds the interface if it hasn't been built yet and then launches the Gradio web server to make the interface accessible.

In [ ]:
#| export
def create_chat_app(
        config: ChatAppConfig # Instance from the config.ChatAppConfig module
        ) -> GradioChat:
    """Create a complete chat application from a configuration"""
    base_app = BaseChatApp(config)
    return GradioChat(base_app)

## Example

Below is an example to create a simple chat UI wich follows more or less the styling confentions from Waterschap Drents Overijsselse Delta.

In [ ]:
#|eval: false
# Eval is false to prevent testing when nbdev_test or nbdev_prepare is run. The api_key is stored in a .env file and that is not accessible at test time.
themeWDODelta = gr.themes.Base(
    primary_hue=gr.themes.Color(c100="#ffedd5", c200="#ffddb3", c300="#fdba74", c400="#f29100", c50="#fff7ed", c500="#f97316", c600="#ea580c", c700="#c2410c", c800="#9a3412", c900="#7c2d12", c950="#6c2e12"),
    neutral_hue="slate",
    radius_size="sm",
    font=['VivalaSansRound', 'ui-sans-serif', 'system-ui', 'sans-serif'],
).set(
    embed_radius='*radius_xs',
    border_color_accent='*primary_400',
    border_color_accent_dark='*secondary_700',
    border_color_primary='*secondary_700',
    border_color_primary_dark='*secondary_700',
    color_accent='*primary_400',
    shadow_drop='*shadow_drop_lg',
    button_primary_background_fill='*primary_400',
    button_primary_background_fill_dark='*primary_400',
    button_primary_background_fill_hover='*secondary_700',
    button_primary_background_fill_hover_dark='*secondary_700',
    button_primary_border_color='*secondary_700',
    button_primary_border_color_dark='*secondary_700'
)

# Create a test configuration
test_config = ChatAppConfig(
    app_name="Job Description Assistant",
    description="Chat with an AI to create better job descriptions",
    system_prompt="You are an assistant that helps users create professional job descriptions. Ask questions to gather information about the position and responsibilities.",
    starter_prompt="Hello! I'm your job description assistant. Tell me about the position you'd like to create a description for.",
    model=ModelConfig(
        provider="togetherai",
        model_name="mistralai/Mistral-7B-Instruct-v0.3",
        api_key_env_var="TG_API_KEY"
    ),
    theme=themeWDODelta,
    logo_path=Path("../data/wdod_logo.svg")
)

# Create and launch the app
app = create_chat_app(test_config)
app.launch(share=False, # Set share=False if you don't want a public URL
        pwa=True # Set pwa=False if you don't want a progressive web app.
        )

ValueError: The environment variable TG_API_KEY is not found in the .env file.

Close all Gradio clients and ports

In [ ]:
#gr.close_all()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()